In [1]:
import sys
sys.path.insert(0, "./../")

In [2]:
import os
from app.mongodb_engine import MongoDB
from app.processor import PDFToSentenceEmbedding

In [3]:
import dotenv

In [4]:
dotenv.load_dotenv()

True

In [5]:
embedding_generator = PDFToSentenceEmbedding()

In [6]:
mongo_db_engine = MongoDB()

In [7]:
question = "What are steps to take when finding projects to build your experience?"

In [8]:
file_name = "eBook-How-to-Build-a-Career-in-AI.pdf"

In [9]:
query_vector = embedding_generator.model.encode(question).tolist()

In [10]:
retrieved_results = mongo_db_engine.vector_search(
    query_vector=query_vector, file_name=file_name)

In [26]:
context_dict = [{"sentence": item['text'], "page number": item['pageLabel']}
                for item in retrieved_results]

In [27]:
import json

In [30]:
print(json.dumps(context_dict))

[
    {
        "sentence": "PAGE 18It goes without saying that we should only work on projects that are responsible, ethical, and \nbeneficial to people. But those limits leave a large variety to choose from. In the previous chapter, \nI wrote about how to identify and scope AI projects. This chapter and the next have a slightly \ndifferent emphasis: picking and executing projects with an eye toward career development.\nA fruitful career will include many projects, hopefully growing in scope, complexity, and impact \nover time. Thus, it is fine to start small. Use early projects to learn and gradually step up to \nbigger projects as your skills grow.\nWhen you\u2019re starting out, don\u2019t expect others to hand great ideas or resources to you on a platter. \nMany people start by working on small projects in their spare time. With initial successes \u2014 even \nsmall ones \u2014 under your belt, your growing skills increase your ability to come up with better \nideas, and it become

In [162]:
def generate_prompt(retrieved_results):

    context_dict = [{"sentence": item['text'], "page number": item['pageLabel']}
                    for item in retrieved_results]

    context_json = json.dumps(context_dict)

    prompt = "Given retrieved document content with page number in json format, please generate a concise and accurate answer to the following question with a page number to indicate where the evidence is. The answer should follow below template: Answer: {Answer} Page Number->{page_number}." + \
        f"\n {context_json}"
    return prompt

In [163]:
prompt = generate_prompt(retrieved_results)

In [164]:

print(prompt)

Given retrieved document content with page number in json format, please generate a concise and accurate answer to the following question with a page number to indicate where the evidence is. The answer should follow below template: Answer: {Answer} Page Number->{page_number}.
 [{"sentence": "PAGE 18It goes without saying that we should only work on projects that are responsible, ethical, and \nbeneficial to people. But those limits leave a large variety to choose from. In the previous chapter, \nI wrote about how to identify and scope AI projects. This chapter and the next have a slightly \ndifferent emphasis: picking and executing projects with an eye toward career development.\nA fruitful career will include many projects, hopefully growing in scope, complexity, and impact \nover time. Thus, it is fine to start small. Use early projects to learn and gradually step up to \nbigger projects as your skills grow.\nWhen you\u2019re starting out, don\u2019t expect others to hand great idea

In [165]:
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  # for OpenAI API callsa

In [166]:
completion = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {'role': 'system',
         'content': prompt,
         },
        {"role": "user",
         "content": question}
    ],
    temperature=0,
    stream=False  # this time, we set stream=True
)

In [167]:
print(completion.choices[0].message.content)

Answer: The steps to take when finding projects to build your experience are as follows:
1. Identify a business problem (not an AI problem) and ask domain experts about the top three things they wish worked better. This helps you understand the real-world challenges that can be addressed with AI. (Page Number->15)
2. Brainstorm potential AI solutions for the identified problem. Consider different approaches and technologies that can be used to solve the problem effectively. (Page Number->15)
3. Join existing projects if someone else has an idea that aligns with your interests and goals. Collaborating with others can provide valuable learning opportunities and help you contribute to meaningful projects. (Page Number->18)
4. Keep reading, taking courses, and talking to domain experts. Continuous learning and exposure to new ideas can spark creativity and help you generate project ideas. (Page Number->18)
5. Focus on a specific application area where machine learning has not yet been exte

In [168]:
string = completion.choices[0].message.content

In [ ]:
get_

In [181]:
import re

pattern = "Page Number->(\d+)"
# Find all matches of the pattern in the string
founds = re.findall(pattern, string)

In [183]:
if founds:
    page_number = int(founds[0])

In [185]:
def get_pag_number(string):

    page_number = 1
    pattern = "Page Number->(\d+)"
    # Find all matches of the pattern in the string
    founds = re.findall(pattern, string)

    if founds:
        try:
            page_number = int(founds[0])
        except IndexError:
            pass

    return page_number

In [186]:
get_pag_number(string)

15

In [182]:
founds

['15', '15', '18', '18', '18']

In [108]:
print(completion.choices[0].message.content)

Answer: The steps to take when finding projects to build your experience are as follows:
1. Identify a business problem (not an AI problem) and ask domain experts about the top three things they wish worked better. (Page Number: 15)
2. Brainstorm potential AI solutions for the identified problem. (Page Number: 15)
3. Join existing projects by collaborating with someone who already has an idea. (Page Number: 18)
4. Keep reading, taking courses, and talking to domain experts to generate new project ideas. (Page Number: 18)
5. Focus on an application area that has not yet been explored with machine learning. (Page Number: 18)

Page Number: (15, 18)


In [99]:
import re
isfound = re.findall(pattern="Page Number: ((.*))",
                     string=completion.choices[0].message.content)

In [100]:
isfound[0]

('15)', '15)')

In [140]:
string = """
Answer: The steps to take when finding projects to build your experience are as follows:
1. Identify a business problem (not an AI problem) and ask domain experts about the top three things they wish worked better. (Page Number: 15)
2. Brainstorm potential AI solutions for the identified problem. (Page Number: 15)
3. Join existing projects by collaborating with someone who already has an idea. (Page Number: 18)
4. Keep reading, taking courses, and talking to domain experts to generate new project ideas. (Page Number: 18)
5. Focus on an application area that has not yet been explored with machine learning. (Page Number: 18)

Page Number->15
"""

In [141]:
import re

# string = "Page Number: (15, 18)"

# Define the regular expression pattern
pattern = "Page Number->(*)"

# Find all matches of the pattern in the string
matches = re.findall(pattern, string)

# Extract the numbers from the matches
numbers = [match for match in matches[0]]

# Print the numbers
print(numbers)

error: nothing to repeat at position 14

In [ ]:
retrieved_results

[{'fileName': 'eBook-How-to-Build-a-Career-in-AI.pdf',
  'textIdx': 17,
  'pageLabel': '18',
  'text': 'PAGE 18It goes without saying that we should only work on projects that are responsible, ethical, and \nbeneficial to people. But those limits leave a large variety to choose from. In the previous chapter, \nI wrote about how to identify and scope AI projects. This chapter and the next have a slightly \ndifferent emphasis: picking and executing projects with an eye toward career development.\nA fruitful career will include many projects, hopefully growing in scope, complexity, and impact \nover time. Thus, it is fine to start small. Use early projects to learn and gradually step up to \nbigger projects as your skills grow.\nWhen you’re starting out, don’t expect others to hand great ideas or resources to you on a platter. \nMany people start by working on small projects in their spare time. With initial successes — even \nsmall ones — under your belt, your growing skills increase you

In [ ]:
retrieved_results

[{'fileName': 'eBook-How-to-Build-a-Career-in-AI.pdf',
  'textIdx': 17,
  'pageLabel': '18',
  'text': 'PAGE 18It goes without saying that we should only work on projects that are responsible, ethical, and \nbeneficial to people. But those limits leave a large variety to choose from. In the previous chapter, \nI wrote about how to identify and scope AI projects. This chapter and the next have a slightly \ndifferent emphasis: picking and executing projects with an eye toward career development.\nA fruitful career will include many projects, hopefully growing in scope, complexity, and impact \nover time. Thus, it is fine to start small. Use early projects to learn and gradually step up to \nbigger projects as your skills grow.\nWhen you’re starting out, don’t expect others to hand great ideas or resources to you on a platter. \nMany people start by working on small projects in their spare time. With initial successes — even \nsmall ones — under your belt, your growing skills increase you

In [ ]:
impo

In [20]:
contents = []
for chunk in completion:
    print(chunk)
    print(chunk.choices[0].delta.content)
    contents.append(chunk.choices[0].delta.content)
    # print("****************")

ChatCompletionChunk(id='chatcmpl-8ZOir71ajdlfmAYaP1ZcVCkpOc7qd', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1703447205, model='gpt-3.5-turbo-0613', object='chat.completion.chunk', system_fingerprint=None)

ChatCompletionChunk(id='chatcmpl-8ZOir71ajdlfmAYaP1ZcVCkpOc7qd', choices=[Choice(delta=ChoiceDelta(content='2', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1703447205, model='gpt-3.5-turbo-0613', object='chat.completion.chunk', system_fingerprint=None)
2
ChatCompletionChunk(id='chatcmpl-8ZOir71ajdlfmAYaP1ZcVCkpOc7qd', choices=[Choice(delta=ChoiceDelta(content=None, function_call=None, role=None, tool_calls=None), finish_reason='stop', index=0, logprobs=None)], created=1703447205, model='gpt-3.5-turbo-0613', object='chat.completion.chunk', system_fingerprint=None)
None


In [21]:
contents

['', '2', None]

In [29]:
import openai

# Set up your OpenAI API credentials
openai.api_key = 'YOUR_API_KEY'

# Define the prompt for your RAG application
prompt = "Given the top 5 retrieved document sentences, please generate a concise and accurate answer to the following question: [question]"

# Define the question you want to ask
question = "What is the capital city of France?"

# Define the retrieved document sentences
retrieved_sentences = [
    "Paris is the capital and largest city of France.",
    "The city of Paris is known for its iconic landmarks such as the Eiffel Tower and Louvre Museum.",
    "Paris is often referred to as the 'City of Love' and is famous for its romantic atmosphere.",
    "The Seine River flows through the city of Paris, adding to its beauty.",
    "French is the official language spoken in Paris and throughout the country."
]

# Combine the prompt, question, and retrieved sentences
combined_prompt = f"{prompt}\n\nQuestion: {question}\nRetrieved Sentences:\n{retrieved_sentences[0]}\n{retrieved_sentences[1]}\n{retrieved_sentences[2]}\n{retrieved_sentences[3]}\n{retrieved_sentences[4]}\n"

# Generate the answer using GPT-3
response = openai.Completion.create(
    engine="text-davinci-003",
    prompt=combined_prompt,
    max_tokens=100,
    n=1,
    stop=None,
    temperature=0.7,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0
)

# Extract and print the generated answer
answer = response.choices[0].text.strip()
print("Generated Answer:", answer)

APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
